# Imports

In [1]:
import os
os.chdir("D:\PulpitE\FPL_ML")

In [2]:
import pandas as pd
import numpy as np
from vaastav.getters import *
from datetime import datetime

# Constants

In [3]:
NEXT_GW = 25

# fixtures in sample
FIS = 8

CURRENT_SEASON = '2022-23'

# Upcoming fixtures

In [4]:
fixtures = get_fixtures_data()

In [5]:
df_fixtures = pd.DataFrame(fixtures)

In [6]:
df_upcoming_fixtures = df_fixtures[df_fixtures["event"] >= NEXT_GW]

In [7]:
def team_id_to_name(team_id):
    teams = ['Arsenal',
     'Aston Villa',
     'Bournemouth',
     'Brentford',
     'Brighton',
     'Chelsea',
     'Crystal Palace',
     'Everton',
     'Fulham',
     'Leicester City',
     'Leeds United',
     'Liverpool',
     'Manchester City',
     'Manchester Utd',
     'Newcastle Utd',
     "Nott'ham Forest",
     'Southampton',
     'Tottenham',
     'West Ham',
     'Wolves']
    return teams[team_id - 1]

In [8]:
df_upcoming_fixtures["Squad H"] = df_upcoming_fixtures["team_h"].apply(team_id_to_name)
df_upcoming_fixtures["Squad A"] = df_upcoming_fixtures["team_a"].apply(team_id_to_name)

C:\Users\user\AppData\Local\Temp\ipykernel_17332\3776683092.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_upcoming_fixtures["Squad H"] = df_upcoming_fixtures["team_h"].apply(team_id_to_name)
C:\Users\user\AppData\Local\Temp\ipykernel_17332\3776683092.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_upcoming_fixtures["Squad A"] = df_upcoming_fixtures["team_a"].apply(team_id_to_name)


In [9]:
df_upcoming_fixtures.rename(columns={"event": "gw"}, inplace=True)

C:\users\user\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [10]:
df_upcoming_fixtures

,code,gw,finished,finished_provisional,id,kickoff_time,minutes,provisional_start_time,started,team_a,team_a_score,team_h,team_h_score,stats,team_h_difficulty,team_a_difficulty,pulse_id,Squad H,Squad A
232,2293053,25.0,False,False,244,2023-02-24T20:00:00Z,0,False,False,20,NaN,9,NaN,[],2,2,75154,Fulham,Wolves
233,2293052,25.0,False,False,243,2023-02-25T15:00:00Z,0,False,False,2,NaN,8,NaN,[],2,2,75153,Everton,Aston Villa
234,2293054,25.0,False,False,245,2023-02-25T15:00:00Z,0,False,False,17,NaN,11,NaN,[],2,2,75155,Leeds United,Southampton
235,2293055,25.0,False,False,246,2023-02-25T15:00:00Z,0,False,False,1,NaN,10,NaN,[],4,3,75156,Leicester City,Arsenal
236,2293059,25.0,False,False,250,2023-02-25T15:00:00Z,0,False,False,16,NaN,19,NaN,[],2,3,75160,West Ham,Nott'ham Forest
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2293185,38.0,False,False,376,2023-05-28T15:30:00Z,0,False,False,3,NaN,8,NaN,[],2,2,75286,Everton,Bournemouth
376,2293186,38.0,False,False,377,2023-05-28T15:30:00Z,0,False,False,18,NaN,11,NaN,[],3,2,75287,Leeds United,Tottenham
377,2293187,38.0,False,False,378,2023-05-28T15:30:00Z,0,False,False,19,NaN,10,NaN,[],2,3,75288,Leicester City,West Ham
378,2293188,38.0,False,False,379,2023-05-28T15:30:00Z,0,False,False,9,NaN,14,NaN,[],2,4,75289,Manchester Utd,Fulham


In [11]:
df_upcoming_fixtures = df_upcoming_fixtures[["gw", "kickoff_time", "Squad H", "Squad A"]]

In [12]:
df_upcoming_fixtures["kickoff_time"] = df_upcoming_fixtures["kickoff_time"].str[:10]

C:\Users\user\AppData\Local\Temp\ipykernel_17332\436943387.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_upcoming_fixtures["kickoff_time"] = df_upcoming_fixtures["kickoff_time"].str[:10]


In [13]:
df_upcoming_fixtures.head(15)

,gw,kickoff_time,Squad H,Squad A
232,25.0,2023-02-24,Fulham,Wolves
233,25.0,2023-02-25,Everton,Aston Villa
234,25.0,2023-02-25,Leeds United,Southampton
235,25.0,2023-02-25,Leicester City,Arsenal
236,25.0,2023-02-25,West Ham,Nott'ham Forest
237,25.0,2023-02-25,Bournemouth,Manchester City
238,25.0,2023-02-25,Crystal Palace,Liverpool
239,25.0,2023-02-26,Tottenham,Chelsea
240,25.0,2023-03-01,Arsenal,Everton
241,25.0,2023-03-01,Liverpool,Wolves


# Active players

In [14]:
def get_active_players():
    previous_fixtures = pd.read_csv("data/final_dataset.csv")
    name_counts = previous_fixtures.groupby('Name').size()
    filtered_names = name_counts[name_counts >= FIS].index
    result = previous_fixtures[previous_fixtures['Name'].isin(filtered_names)].groupby('Name').tail(1)
    result = result.loc[:, ~result.columns.str.contains('^Unnamed')]
    return result

In [15]:
active_players = get_active_players()

In [16]:
active_players[["Name", "Squad"]]

,Name,Squad
18,Brenden-Aaronson,Leeds United
38,Che-Adams,Southampton
55,Tyler-Adams,Leeds United
76,Tosin-Adarabioyo,Fulham
101,Rayan-Ait-Nouri,Wolves
...,...,...
7392,Jordan-Zemura,Bournemouth
7404,Oleksandr-Zinchenko,Arsenal
7422,Hakim-Ziyech,Chelsea
7438,Kurt-Zouma,West Ham


In [17]:
picked_columns = ["Name", "gw", "kickoff_time", "Squad", "Squad H", "Squad A", "Was Home", "Opponent"]

In [18]:
upcoming_player_fixtures_home = active_players.rename(columns={"Squad": "Squad H"}).set_index("Squad H").join(df_upcoming_fixtures.set_index("Squad H")).reset_index(level=0)
upcoming_player_fixtures_home["Squad"] = upcoming_player_fixtures_home["Squad H"]
upcoming_player_fixtures_home["Was Home"] = True
upcoming_player_fixtures_home["Opponent"] = upcoming_player_fixtures_home["Squad A"]
upcoming_player_fixtures_home = upcoming_player_fixtures_home[picked_columns]

In [19]:
upcoming_player_fixtures_away = active_players.rename(columns={"Squad": "Squad A"}).set_index("Squad A").join(df_upcoming_fixtures.set_index("Squad A")).reset_index(level=0)
upcoming_player_fixtures_away["Squad"] = upcoming_player_fixtures_away["Squad A"]
upcoming_player_fixtures_away["Was Home"] = False
upcoming_player_fixtures_away["Opponent"] = upcoming_player_fixtures_away["Squad H"]
upcoming_player_fixtures_away = upcoming_player_fixtures_away[picked_columns]

In [20]:
upcoming_player_fixtures = pd.concat([upcoming_player_fixtures_home, upcoming_player_fixtures_away]).sort_values(by=["Name", "gw"])

In [21]:
upcoming_player_fixtures["Season"] = CURRENT_SEASON
upcoming_player_fixtures["Finished"] = False

In [22]:
upcoming_player_fixtures = upcoming_player_fixtures.rename(columns={"kickoff_time": "Date"})

In [23]:
upcoming_player_fixtures[upcoming_player_fixtures["Name"] == "Gabriel-Dos-Santos"]

,Name,gw,Date,Squad,Squad H,Squad A,Was Home,Opponent,Season,Finished
0,Gabriel-Dos-Santos,25.0,2023-03-01,Arsenal,Arsenal,Everton,True,Everton,2022-23,False
0,Gabriel-Dos-Santos,25.0,2023-02-25,Arsenal,Leicester City,Arsenal,False,Leicester City,2022-23,False
1,Gabriel-Dos-Santos,26.0,2023-03-04,Arsenal,Arsenal,Bournemouth,True,Bournemouth,2022-23,False
1,Gabriel-Dos-Santos,27.0,2023-03-12,Arsenal,Fulham,Arsenal,False,Fulham,2022-23,False
2,Gabriel-Dos-Santos,28.0,2023-03-19,Arsenal,Arsenal,Crystal Palace,True,Crystal Palace,2022-23,False
3,Gabriel-Dos-Santos,29.0,2023-04-01,Arsenal,Arsenal,Leeds United,True,Leeds United,2022-23,False
2,Gabriel-Dos-Santos,30.0,2023-04-09,Arsenal,Liverpool,Arsenal,False,Liverpool,2022-23,False
3,Gabriel-Dos-Santos,31.0,2023-04-16,Arsenal,West Ham,Arsenal,False,West Ham,2022-23,False
4,Gabriel-Dos-Santos,32.0,2023-04-21,Arsenal,Arsenal,Southampton,True,Southampton,2022-23,False
4,Gabriel-Dos-Santos,33.0,2023-04-26,Arsenal,Manchester City,Arsenal,False,Manchester City,2022-23,False


In [24]:
upcoming_player_fixtures.shape

(6422, 10)

# Loading previous fixtures

In [25]:
previous_fixtures = pd.read_csv("data/final_dataset.csv")

In [26]:
previous_fixtures["Finished"] = True

# Assigning team elo

In [27]:
team_ratings = pd.read_csv("data/team_ratings_history.csv")

In [28]:
# function to assign rating for team
def get_rating_for_team(team_ratings, team_name, date):
    rating = None
    try:
        rating = team_ratings[(team_ratings['From'] <= date.strftime("%Y-%m-%d")) & (team_ratings['Club'] == team_name)].iloc[-1]['Elo']
    except:
        print(team_name, date)
    return rating

In [29]:
def fbref_to_team_elo(team_name):
    # changes name of a team
    if team_name == "Leicester City":
        return "Leicester"
    if team_name == "Manchester City":
        return "Man City"
    if team_name == "Manchester Utd":
        return "Man United"
    if team_name == "Newcastle Utd":
        return "Newcastle"
    if team_name == "Nott'ham Forest":
        return "Forest"
    if team_name == "Leeds United":
        return "Leeds"
    return team_name

In [30]:
%%time
upcoming_player_fixtures["Opp rating"] = upcoming_player_fixtures.apply(lambda row : get_rating_for_team(team_ratings,
                     fbref_to_team_elo(row['Opponent']), datetime.strptime(row['Date'], '%Y-%m-%d')), axis=1)

CPU times: total: 1min 9s
Wall time: 1min 10s


In [31]:
%%time
upcoming_player_fixtures["Team rating"] = upcoming_player_fixtures.apply(lambda row : get_rating_for_team(team_ratings,
                     fbref_to_team_elo(row['Squad']), datetime.strptime(row['Date'], '%Y-%m-%d')), axis=1)

CPU times: total: 1min 13s
Wall time: 1min 15s


In [32]:
upcoming_player_fixtures[upcoming_player_fixtures["Name"] == "Gabriel-Dos-Santos"]

,Name,gw,Date,Squad,Squad H,Squad A,Was Home,Opponent,Season,Finished,Opp rating,Team rating
0,Gabriel-Dos-Santos,25.0,2023-03-01,Arsenal,Arsenal,Everton,True,Everton,2022-23,False,1667.462280,1946.912109
0,Gabriel-Dos-Santos,25.0,2023-02-25,Arsenal,Leicester City,Arsenal,False,Leicester City,2022-23,False,1746.499023,1946.912109
1,Gabriel-Dos-Santos,26.0,2023-03-04,Arsenal,Arsenal,Bournemouth,True,Bournemouth,2022-23,False,1636.081665,1946.912109
1,Gabriel-Dos-Santos,27.0,2023-03-12,Arsenal,Fulham,Arsenal,False,Fulham,2022-23,False,1717.072754,1946.912109
2,Gabriel-Dos-Santos,28.0,2023-03-19,Arsenal,Arsenal,Crystal Palace,True,Crystal Palace,2022-23,False,1742.434937,1946.912109
3,Gabriel-Dos-Santos,29.0,2023-04-01,Arsenal,Arsenal,Leeds United,True,Leeds United,2022-23,False,1708.115723,1946.912109
2,Gabriel-Dos-Santos,30.0,2023-04-09,Arsenal,Liverpool,Arsenal,False,Liverpool,2022-23,False,1963.465210,1946.912109
3,Gabriel-Dos-Santos,31.0,2023-04-16,Arsenal,West Ham,Arsenal,False,West Ham,2022-23,False,1748.305420,1946.912109
4,Gabriel-Dos-Santos,32.0,2023-04-21,Arsenal,Arsenal,Southampton,True,Southampton,2022-23,False,1651.960449,1946.912109
4,Gabriel-Dos-Santos,33.0,2023-04-26,Arsenal,Manchester City,Arsenal,False,Manchester City,2022-23,False,2026.310791,1946.912109


# Adding features

In [33]:
%%time
# calculating features for every player
overall_average_columns = ["Min", "Gls", "Ast", "CrdY", "CrdR", "xG", "xAG", "CS", "Team Score", "Opp Score", "FPL"]

for name in upcoming_player_fixtures["Name"].unique():
    for col in overall_average_columns:
        value = previous_fixtures[previous_fixtures["Name"] == name][col].mean()
        upcoming_player_fixtures.loc[upcoming_player_fixtures["Name"] == name, col + "AvgOverall"] = value
        
    upcoming_player_fixtures.loc[upcoming_player_fixtures["Name"] == name, 'ScoreForLast5'] = previous_fixtures[previous_fixtures["Name"] == name]['Team Score'].tail(5).mean()
    upcoming_player_fixtures.loc[upcoming_player_fixtures["Name"] == name, 'ScoreAgainstLast5'] = previous_fixtures[previous_fixtures["Name"] == name]['Opp Score'].tail(5).mean()

CPU times: total: 8.05 s
Wall time: 8.1 s


In [34]:
upcoming_player_fixtures.columns

Index(['Name', 'gw', 'Date', 'Squad', 'Squad H', 'Squad A', 'Was Home',
       'Opponent', 'Season', 'Finished', 'Opp rating', 'Team rating',
       'MinAvgOverall', 'GlsAvgOverall', 'AstAvgOverall', 'CrdYAvgOverall',
       'CrdRAvgOverall', 'xGAvgOverall', 'xAGAvgOverall', 'CSAvgOverall',
       'Team ScoreAvgOverall', 'Opp ScoreAvgOverall', 'FPLAvgOverall',
       'ScoreForLast5', 'ScoreAgainstLast5'],
      dtype='object')

# Concat dataframes

In [35]:
all_fixtures = pd.concat([previous_fixtures, upcoming_player_fixtures])

In [38]:
all_fixtures.tail(20)

1727    False
1770    False
1728    False
1771    False
1729    False
1772    False
2721    False
2719    False
2722    False
2720    False
2721    False
2723    False
2724    False
2722    False
2725    False
2723    False
2726    False
2724    False
2727    False
2725    False
Name: Finished, dtype: bool

In [37]:
all_fixtures.to_csv("data/upcoming_fixtures.csv")